# 쿠팡 데이터 크롤링

___

## Global functions, parameters

In [118]:
import requests
import bs4
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import TimeoutException
import time

driver = webdriver.Chrome()

In [2]:
def getURL(seed, params=None, select='a', num_retries=2):
    html = requests.get(seed, params=params, headers=headers)
    
    if 500<=html.status_code<600 and num_retries>0:
        print(html.status_code, html.reason)
        return getURL(seed, params, num_retries-1)
    
    dom = BeautifulSoup(html.text, 'lxml')
    links = dom.select(select)
    
    return ['http://www.coupang.com' + link['href'] for link in links if link.has_attr('href')]

In [3]:
headers = {
    "user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36",
    "accept-encoding":"false"
}

___

### 과자
1. link
2. img
3. brand
4. title: 제품명(있는 그대로)
5. product: 보여줄 제품명(브랜드 + 연령대 + 형태)
6. price: 가격(int)
7. rating: 평점
8. replyCount: 상품평 갯수
9. reply: 20개

In [4]:
params_snack = {
    "listSize": "120",
    "brand": "",
    "offerCondition": "",
    "filterType": "",
    "isPriceRange": "false",
    "minPrice": "",
    "maxPrice": "",
    "page": "1",
    "channel": "user",
    "fromComponent": "Y",
    "selectedPlpKeepFilter": "",
    "sorter": "bestAsc",
    "filter": "",
    "component": "221916",
    "rating": "0",
}

In [5]:
seed_snack = 'http://www.coupang.com/np/categories/222016'

In [6]:
links_snack = []
links_snack.extend(getURL(seed_snack, params_snack, select='ul.baby-product-list a.baby-product-link'))
links_snack = links_snack[:100]

In [32]:
snackList = []

In [35]:
def getInfo_snack(url, num_retries=2):
    html = requests.get(url, headers=headers)
    
    if 500<=html.status_code<600 and num_retries>0:
        print(html.status_code, html.reason)
        return getInfo_snack(seed, params, num_retries-1)
    
    dom = BeautifulSoup(html.text, 'lxml')
    if len(dom.select('div.oos-label')) == 0 and int(dom.select('a.moveAnchor span.count')[0].get_text().replace('개 상품평', '').replace(',', '')) > 0:
        link = html.url
        img = 'http:' + dom.select('img.prod-image__detail')[0]['src']
        brand = dom.select('a.prod-brand-name')[0].get_text()
        title = dom.select('h2.prod-buy-header__title')[0].get_text()
        # product = 
        price = int(dom.select('span.total-price')[0].get_text().replace('\n', '').replace('원', '').replace(',', ''))
        replyCount = int(dom.select('a.moveAnchor span.count')[0].get_text().replace('개 상품평', '').replace(',', ''))
        
        driver.get(url)
        body = driver.find_element_by_tag_name('body')
        while True:
            try:
                driver.find_element_by_css_selector('.sdp-review__average__total-star__info-gray div')
                break
            except NoSuchElementException:
                body.send_keys(Keys.PAGE_DOWN)
                driver.implicitly_wait(0.5)
                
        rating = driver.find_element_by_css_selector('.sdp-review__average__total-star__info-gray div')
        rating = rating.get_attribute('data-rating')
        replyResult = []
        buttonList = []
        buttonList.extend(driver.find_elements_by_css_selector('.sdp-review__article__page__num.js_reviewArticlePageBtn'))
        
        def getReply(pagenum):
            try:
                element = WebDriverWait(driver, 5).until(lambda x: x.find_elements_by_css_selector('.js_reviewArticleListContainer article div.sdp-review__article__list__review__content.js_reviewArticleContent')) 
                replyResult.extend(reply.text.replace('\n', ' ') for reply in element)
                time.sleep(1)
            except TimeoutException:
                return None
            
            while True:
                try:
                    buttonList = []
                    buttonList.extend(driver.find_elements_by_css_selector('.sdp-review__article__page__num.js_reviewArticlePageBtn'))
                    break
                except WebDriverException:
                    body.send_keys(Keys.PAGE_DOWN)
                    driver.implicitly_wait(0.5)
            if len(buttonList) > 0:
                buttonList[pagenum].click()
            else:
                return None
        
        if len(buttonList) > 0:
            for i in range(1, 5):
                getReply(i)
                time.sleep(0.5)
    
        result = tuple([link, img, brand, title, price, replyCount, rating, replyResult])
        snackList.append(result)
        return 1
    
    else:
        return 0

In [36]:
for i in range(100):
    if getInfo_snack(links_snack[i]) == 0:
        print('Skip: ' + links_snack[i])
    time.sleep(0.3)

Skip: http://www.coupang.com/vp/products/96477934?itemId=296996220&vendorItemId=3730702348


In [43]:
snackList = snackList[7:]

In [44]:
len(snackList)

99

평점 없는건 제외시켰음.

___

### 분유
1. link
2. img
3. brand
4. origin: 원산지(국내 / 해외)
5. title: 제품명(있는 그대로)
6. product: 보여줄 제품명(브랜드 + 연령대 + 형태)
7. stage: 분유 단계(int)
8. type: 분말 / 액상
9. price: 가격(int)
10. rating: 평점
11. replyCount: 상품평 갯수
12. reply: 20개

In [45]:
params_milkPowder = {
    "listSize": "120",
    "brand": "",
    "offerCondition": "",
    "filterType": "",
    "isPriceRange": "false",
    "minPrice": "",
    "maxPrice": "",
    "page": "1",
    "channel": "user",
    "fromComponent": "N",
    "selectedPlpKeepFilter": "",
    "sorter": "bestAsc",
    "filter": "",
    "component": "221895",
    "rating": "0",
}

In [46]:
seed_milkPowder = 'http://www.coupang.com/np/categories/221995'

In [47]:
links_milkPowder = []
links_milkPowder.extend(getURL(seed_milkPowder, params_milkPowder, select='ul.baby-product-list a.baby-product-link'))
links_milkPowder = links_milkPowder[:100]

In [48]:
milkPowderList = []

In [79]:
def getInfo_milkPowder(url, num_retries=2):
    html = requests.get(url, headers=headers)
    
    if 500<=html.status_code<600 and num_retries>0:
        print(html.status_code, html.reason)
        return getInfo_milkPowder(seed, params, num_retries-1)
    
    dom = BeautifulSoup(html.text, 'lxml')
    if len(dom.select('div.oos-label')) == 0 and int(dom.select('a.moveAnchor span.count')[0].get_text().replace('개 상품평', '').replace(',', '')) > 0:
        link = html.url
        img = 'http:' + dom.select('img.prod-image__detail')[0]['src']
        brand = dom.select('a.prod-brand-name')[0].get_text()
        title = dom.select('h2.prod-buy-header__title')[0].get_text()
        # product = 
        index = 0
        stage = 0
        for index in range(len(dom.select('li.prod-attr-item'))):
            try:
                stage = int(dom.select('li.prod-attr-item')[index].get_text().replace('\n', '').replace('단계: ', '').replace('단계', ''))
            except ValueError:
                index += 1
            
        if stage == 0:
            stageIndex = title.find('단계')
            try:
                stage = int(title[stageIndex-1])
            except ValueError:
                return 0
        
        if '액상분유' in title:
            milkPowderType = '액상분유'
        elif '스틱분유' in title:
            milkPowderType = '스틱분유'
        else:
            milkPowderType = '원통형분유'
        price = int(dom.select('span.total-price')[0].get_text().replace('\n', '').replace('원', '').replace(',', ''))
        replyCount = int(dom.select('a.moveAnchor span.count')[0].get_text().replace('개 상품평', '').replace(',', ''))
        
        driver.get(url)
        body = driver.find_element_by_tag_name('body')
        while True:
            try:
                driver.find_element_by_css_selector('.sdp-review__average__total-star__info-gray div')
                break
            except NoSuchElementException:
                body.send_keys(Keys.PAGE_DOWN)
                driver.implicitly_wait(0.5)
        
        origin = driver.find_elements_by_css_selector('.prod-delivery-return-policy-table.essential-info-table tbody tr td')[1].text
        rating = driver.find_element_by_css_selector('.sdp-review__average__total-star__info-gray div')
        rating = rating.get_attribute('data-rating')
        replyResult = []
        buttonList = []
        buttonList.extend(driver.find_elements_by_css_selector('.sdp-review__article__page__num.js_reviewArticlePageBtn'))
        
        def getReply(pagenum):
            try:
                element = WebDriverWait(driver, 5).until(lambda x: x.find_elements_by_css_selector('.js_reviewArticleListContainer article div.sdp-review__article__list__review__content.js_reviewArticleContent')) 
                replyResult.extend(reply.text.replace('\n', ' ') for reply in element)
                time.sleep(1)
            except TimeoutException:
                return None
            buttonList = []
            while True:
                try:
                    buttonList.extend(driver.find_elements_by_css_selector('.sdp-review__article__page__num.js_reviewArticlePageBtn'))
                    break
                except WebDriverException:
                    body.send_keys(Keys.PAGE_DOWN)
                    driver.implicitly_wait(0.5)
            if len(buttonList) > 0:
                buttonList[pagenum].click()
            else:
                return None
        
        if len(buttonList) > 0:
            for i in range(1, 5):
                getReply(i)
                time.sleep(0.5)
    
        result = tuple([link, img, brand, origin, title, stage, milkPowderType, price, replyCount, rating, replyResult])
        milkPowderList.append(result)
        return 1
    
    else:
        return 0
    
    return None

In [82]:
for i in range(75, 100, 1):
    if getInfo_milkPowder(links_milkPowder[i]) == 0:
        print('Skip: ' + links_milkPowder[i])
    time.sleep(0.3)

Skip: http://www.coupang.com/vp/products/13355297?itemId=125443118&vendorItemId=3769717414
Skip: http://www.coupang.com/vp/products/116863081?itemId=349863063&vendorItemId=3856161600


In [83]:
len(milkPowderList)

98

2개 스킵: 단계 이상해서, 단계 옵션으로 주어져서 스킵됨.

___

### 장난감
1. link
2. img
3. category: 감각발달완구, 아기체육관, 러닝홈/플레이매트, 아기걸음마, 모빌/딸랑이/치발기, 오뚝이
4. title: 제품명(있는 그대로)
5. product: 보여줄 제품명(브랜드 + 연령대 + 형태)
6. price: 가격(int)
7. rating: 평점
8. replyCount: 상품평 갯수
9. reply: 20개

In [86]:
categoryList_toy = ['감각발달완구', '아기체육관/러닝홈', '아기걸음마', '모빌', '딸랑이', '치발기', '오뚝이']

In [87]:
params_toy = {
    "listSize": "120",
    "brand": "",
    "offerCondition": "",
    "filterType": "",
    "isPriceRange": "false",
    "minPrice": "",
    "maxPrice": "",
    "page": "1",
    "channel": "user",
    "fromComponent": "Y",
    "selectedPlpKeepFilter": "",
    "sorter": "bestAsc",
    "filter": "",
    "component": "",
    "rating": "0",
}

In [88]:
seed_toy_list = ['http://www.coupang.com/np/categories/349712',
                 'http://www.coupang.com/np/categories/349704',
                 'http://www.coupang.com/np/categories/349710',
                 'http://www.coupang.com/np/categories/349687',
                 'http://www.coupang.com/np/categories/349689',
                 'http://www.coupang.com/np/categories/349690',
                 'http://www.coupang.com/np/categories/349694']

In [89]:
toyComponentList = [349612, 349604, 349610, 349587, 349589, 349590, 349594]

In [90]:
links_toy = {
    categoryList_toy[0]: None,
    categoryList_toy[1]: None,
    categoryList_toy[2]: None,
    categoryList_toy[3]: None,
    categoryList_toy[4]: None,
    categoryList_toy[5]: None,
    categoryList_toy[6]: None
}

for i in range(7):
    params_toy['component'] = toyComponentList[i]
    temp = []
    temp.extend(getURL(seed_toy_list[i], params_toy, select='ul.baby-product-list a.baby-product-link'))
    temp = temp[:100]
    links_toy[categoryList_toy[i]] = temp

In [91]:
for (k, v) in links_toy.items():
    print(k, len(v))

감각발달완구 100
아기체육관/러닝홈 100
아기걸음마 100
모빌 100
딸랑이 100
치발기 100
오뚝이 100


In [97]:
links_toy[categoryList_toy[0]]

['http://www.coupang.com/vp/products/5174059?itemId=23450011&vendorItemId=3034349958',
 'http://www.coupang.com/vp/products/1556548?itemId=198924&vendorItemId=3000132612',
 'http://www.coupang.com/vp/products/1526123?itemId=4398101&vendorItemId=3005327603',
 'http://www.coupang.com/vp/products/2515277?itemId=11587748&vendorItemId=3017528256',
 'http://www.coupang.com/vp/products/655132?itemId=2333435&vendorItemId=3000068366',
 'http://www.coupang.com/vp/products/22958568?itemId=89183561&vendorItemId=3154013313',
 'http://www.coupang.com/vp/products/102306?itemId=199082&vendorItemId=3000140857',
 'http://www.coupang.com/vp/products/183499?itemId=334548&vendorItemId=3000244319',
 'http://www.coupang.com/vp/products/392215?itemId=1010744&vendorItemId=3000925179',
 'http://www.coupang.com/vp/products/3339629?itemId=12828011&vendorItemId=3014175068',
 'http://www.coupang.com/vp/products/200049?itemId=369536&vendorItemId=3000275216',
 'http://www.coupang.com/vp/products/66294953?itemId=22310

In [117]:
toyList = []

In [111]:
def getInfo_toy(url, category, num_retries=2):
    html = requests.get(url, headers=headers)
    
    if 500<=html.status_code<600 and num_retries>0:
        print(html.status_code, html.reason)
        return getInfo_toy(seed, params, num_retries-1)
    
    dom = BeautifulSoup(html.text, 'lxml')
    if len(dom.select('div.oos-label')) == 0 and int(dom.select('a.moveAnchor span.count')[0].get_text().replace('개 상품평', '').replace(',', '')) > 0:
        link = html.url
        img = 'http:' + dom.select('img.prod-image__detail')[0]['src']
        category = category
        title = dom.select('h2.prod-buy-header__title')[0].get_text()
        # product = 
        price = int(dom.select('span.total-price')[0].get_text().replace('\n', '').replace('원', '').replace(',', ''))
        replyCount = int(dom.select('a.moveAnchor span.count')[0].get_text().replace('개 상품평', '').replace(',', ''))
        
        driver.get(url)
        body = driver.find_element_by_tag_name('body')
        while True:
            try:
                driver.find_element_by_css_selector('.sdp-review__average__total-star__info-gray div')
                break
            except NoSuchElementException:
                body.send_keys(Keys.PAGE_DOWN)
                driver.implicitly_wait(1)
        
        rating = driver.find_element_by_css_selector('.sdp-review__average__total-star__info-gray div')
        rating = rating.get_attribute('data-rating')
        replyResult = []
        buttonList = []
        buttonList.extend(driver.find_elements_by_css_selector('.sdp-review__article__page__num.js_reviewArticlePageBtn'))
        
        def getReply(pagenum):
            try:
                element = WebDriverWait(driver, 5).until(lambda x: x.find_elements_by_css_selector('.js_reviewArticleListContainer article div.sdp-review__article__list__review__content.js_reviewArticleContent')) 
                replyResult.extend(reply.text.replace('\n', ' ') for reply in element)
                time.sleep(1)
            except TimeoutException:
                return None
            buttonList = []
            while True:
                try:
                    buttonList.extend(driver.find_elements_by_css_selector('.sdp-review__article__page__num.js_reviewArticlePageBtn'))
                    break
                except WebDriverException:
                    body.send_keys(Keys.PAGE_DOWN)
                    driver.implicitly_wait(1)
            if len(buttonList) > 0:
                buttonList[pagenum].click()
            else:
                return None
        
        if len(buttonList) > 0:
            for i in range(1, 5):
                getReply(i)
                time.sleep(0.5)
    
        result = tuple([link, img, category, title, price, replyCount, rating, replyResult])
        toyList.append(result)
        return 1
    
    else:
        return 0
    
    return None

In [115]:
getInfo_toy(links_toy[categoryList_toy[0]][10], categoryList_toy[0])

1

In [112]:
for link in links_toy[categoryList_toy[0]]:
    getInfo_toy(link, categoryList_toy[0])

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=68.0.3440.106)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.13.4 x86_64)


In [120]:
for i in range(7):
    for link in links_toy[categoryList_toy[i]]:
        getInfo_toy(link, categoryList_toy[i])

WebDriverException: Message: unknown error: Element <button class="sdp-review__article__page__num  js_reviewArticlePageBtn" data-page="...">3</button> is not clickable at point (516, 644). Other element would receive the click: <div class="sdp-review__white-dimmed js_reviewWhiteBgDimmed sdp-review__white-dimmed-active"></div>
  (Session info: chrome=68.0.3440.106)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.13.4 x86_64)


In [121]:
len(toyList)

20

___

## 기저귀
1. link
2. img
3. brand
4. origin: 원산지(국내 / 해외)
5. title: 제품명(있는 그대로)
6. product: 보여줄 제품명(브랜드 + 연령대 + 형태)
7. stage: {“신생아”:1,  “소형”:2, “중형”:3, “대형”:4, “특대형”:5, “점보”:6}
8. type: 밴드 / 팬티
9. price: 가격(int)
10. rating: 평점
11. replyCount: 상품평 갯수
12. reply: 20개

In [89]:
size_filter_diaper_boy = {
    '신생아': '1%23attr_11880%244488attr_41%242%40DEFAULT',
    '소형': '1%23attr_11880%244489attr_41%242%40DEFAULT',
    '중형': '1%23attr_11880%244490attr_41%242%40DEFAULT',
    '대형': '1%23attr_11880%244491attr_41%242%40DEFAULT',
    '특대': '1%23attr_11880%244492attr_41%242%40DEFAULT',
    '점보': '1%23attr_11880%244493attr_41%242%40DEFAULT'
}

In [90]:
size_filter_diaper_girl = {
    '신생아': '1%23attr_11880%244488attr_41%241%40DEFAULT',
    '소형': '1%23attr_11880%244489attr_41%241%40DEFAULT',
    '중형': '1%23attr_11880%244490attr_41%241%40DEFAULT',
    '대형': '1%23attr_11880%244491attr_41%241%40DEFAULT',
    '특대': '1%23attr_11880%244492attr_41%241%40DEFAULT',
    '점보': '1%23attr_11880%244493attr_41%241%40DEFAULT'
}

In [91]:
size_filter_diaper_uni = {
    '신생아': '1%23attr_11880%244488attr_41%246%40DEFAULT',
    '소형': '1%23attr_11880%244489attr_41%246%40DEFAULT',
    '중형': '1%23attr_11880%244490attr_41%246%40DEFAULT',
    '대형': '1%23attr_11880%244491attr_41%246%40DEFAULT',
    '특대': '1%23attr_11880%244492attr_41%246%40DEFAULT',
    '점보': '1%23attr_11880%244493attr_41%246%40DEFAULT'
}

In [92]:
sizeList_diaper = ['신생아', '소형', '중형', '대형', '특대', '점보']

In [93]:
params_diaper = {
    "listSize": "60",
    "brand": "",
    "offerCondition": "",
    "filterType": "",
    "isPriceRange": "false",
    "minPrice": "",
    "maxPrice": "",
    "page": "",
    "channel": "user",
    "fromComponent": "N",
    "selectedPlpKeepFilter": "",
    "sorter": "bestAsc",
    "filter": "",
    "component": "221863",
    "rating": "0",
}

In [94]:
seed_diaper = 'http://www.coupang.com/np/categories/221963'

In [95]:
result_diaper_boy = {
    sizeList_diaper[0]: None,
    sizeList_diaper[1]: None,
    sizeList_diaper[2]: None,
    sizeList_diaper[3]: None,
    sizeList_diaper[4]: None,
    sizeList_diaper[5]: None
}

for i in range(6):
    currentpage = 1
    params_diaper['filter'] = size_filter_diaper_boy[sizeList_diaper[i]]
    temp = []
    
    while True:
        params_diaper['page'] = str(currentpage)
        print(params_diaper['filter'])
        if len(getItemNum(seed_diaper, params_diaper)) == 0:
            break
        print(params_diaper['page'])
        temp.extend(getURL(seed_diaper, params_diaper, select='ul.baby-product-list a.baby-product-link'))
        currentpage += 1
        
    result_diaper_boy[sizeList_diaper[i]] = temp

1%23attr_11880%244488attr_41%242%40DEFAULT
http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244488attr_41%242%40DEFAULT&component=221863&rating=0
1
1%23attr_11880%244488attr_41%242%40DEFAULT
http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244488attr_41%242%40DEFAULT&component=221863&rating=0
1%23attr_11880%244489attr_41%242%40DEFAULT
http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244489attr_41%242%40DEFAULT&component=221863&rating=0
1
1%23attr_11880%244489attr_4

In [96]:
for (k, v) in result_diaper_boy.items():
    print(k, len(v))

신생아 60
소형 120
중형 180
대형 420
특대 300
점보 120


In [97]:
result_diaper_girl = {
    sizeList_diaper[0]: None,
    sizeList_diaper[1]: None,
    sizeList_diaper[2]: None,
    sizeList_diaper[3]: None,
    sizeList_diaper[4]: None,
    sizeList_diaper[5]: None
}

for i in range(6):
    currentpage = 1
    params_diaper['filter'] = size_filter_diaper_girl[sizeList_diaper[i]]
    temp = []
    
    while True:
        params_diaper['page'] = str(currentpage)
        print(params_diaper['filter'])
        if len(getItemNum(seed_diaper, params_diaper)) == 0:
            break
        print(params_diaper['page'])
        temp.extend(getURL(seed_diaper, params_diaper, select='ul.baby-product-list a.baby-product-link'))
        currentpage += 1
        
    result_diaper_girl[sizeList_diaper[i]] = temp

1%23attr_11880%244488attr_41%241%40DEFAULT
http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244488attr_41%241%40DEFAULT&component=221863&rating=0
1
1%23attr_11880%244488attr_41%241%40DEFAULT
http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244488attr_41%241%40DEFAULT&component=221863&rating=0
1%23attr_11880%244489attr_41%241%40DEFAULT
http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244489attr_41%241%40DEFAULT&component=221863&rating=0
1
1%23attr_11880%244489attr_4

In [98]:
for (k, v) in result_diaper_girl.items():
    print(k, len(v))

신생아 60
소형 120
중형 180
대형 360
특대 360
점보 120


In [99]:
result_diaper_uni = {
    sizeList_diaper[0]: None,
    sizeList_diaper[1]: None,
    sizeList_diaper[2]: None,
    sizeList_diaper[3]: None,
    sizeList_diaper[4]: None,
    sizeList_diaper[5]: None
}

for i in range(6):
    currentpage = 1
    params_diaper['filter'] = size_filter_diaper_uni[sizeList_diaper[i]]
    temp = []
    
    while True:
        params_diaper['page'] = str(currentpage)
        print(params_diaper['filter'])
        if len(getItemNum(seed_diaper, params_diaper)) == 0:
            break
        print(params_diaper['page'])
        temp.extend(getURL(seed_diaper, params_diaper, select='ul.baby-product-list a.baby-product-link'))
        currentpage += 1
        
    result_diaper_uni[sizeList_diaper[i]] = temp

1%23attr_11880%244488attr_41%246%40DEFAULT
http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244488attr_41%246%40DEFAULT&component=221863&rating=0
1
1%23attr_11880%244488attr_41%246%40DEFAULT
http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244488attr_41%246%40DEFAULT&component=221863&rating=0
2
1%23attr_11880%244488attr_41%246%40DEFAULT
http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=3&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244488attr_41%246%40DEFAULT&component=221863&rating=0
3
1%23attr_11880%244488attr

1
1%23attr_11880%244491attr_41%246%40DEFAULT
http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244491attr_41%246%40DEFAULT&component=221863&rating=0
2
1%23attr_11880%244491attr_41%246%40DEFAULT
http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=3&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244491attr_41%246%40DEFAULT&component=221863&rating=0
3
1%23attr_11880%244491attr_41%246%40DEFAULT
http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=4&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244491attr_41%246%40DEFAULT&component=221863&rating=0
4
1%23attr_11880%244491at

3
1%23attr_11880%244493attr_41%246%40DEFAULT
http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=4&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244493attr_41%246%40DEFAULT&component=221863&rating=0


In [100]:
for (k, v) in result_diaper_uni.items():
    print(k, len(v))

신생아 360
소형 420
중형 540
대형 720
특대 600
점보 180


In [202]:
diaperList = []

In [102]:
def getInfo_diaper(url, sizeInfo, gender, num_retries=2):
    html = requests.get(url, headers=headers)
    
    if 500<=html.status_code<600 and num_retries>0:
        print(html.status_code, html.reason)
        return getInfo_diaper(seed, params, num_retries-1)
    
    dom = BeautifulSoup(html.text, 'lxml')
    if len(dom.select('div.oos-label')) == 0 and len(dom.select('span.total-price strong')) > 0:
        name = dom.select('h2.prod-buy-header__title')[0].get_text()
        brand = dom.select('a.prod-brand-name')[0].get_text()
        price = int(dom.select('span.total-price strong')[0].get_text().replace('\n', '').replace('원', '').replace(',', ''))
        url = html.url
        img = 'http:' + dom.select('img.prod-image__detail')[0]['src']
        diaperList.append(tuple([name, gender, brand, sizeInfo, price, url, img]))
    
    return None

In [163]:
def getall_diaper_boy():
    startTime = time.time()
    for i in range(6):
        for url in result_diaper_boy[sizeList_diaper[i]]:
            getInfo_diaper(url, sizeList_diaper[i], gender_list[1])
    
    print('Done-boy')
    print(time.time() - startTime)
    return None

def getall_diaper_girl():
    startTime = time.time()
    for i in range(6):
        for url in result_diaper_girl[sizeList_diaper[i]]:
            getInfo_diaper(url, sizeList_diaper[i], gender_list[0])
    
    print('Done-girl')
    print(time.time() - startTime)
    return None

def getall_diaper_uni():
    startTime = time.time()
    for i in range(6):
        for url in result_diaper_uni[sizeList_diaper[i]]:
            getInfo_diaper(url, sizeList_diaper[i], gender_list[2])
    
    print('Done-uni')
    print(time.time() - startTime)
    return None

In [203]:
import threading
 
thread_boy = threading.Thread(target=getall_diaper_boy)
thread_girl = threading.Thread(target=getall_diaper_girl)
thread_uni = threading.Thread(target=getall_diaper_uni)

thread_boy.start()
thread_girl.start()
thread_uni.start()

In [227]:
import sqlite3 as sql

In [230]:
con = sql.connect('coupang-bulk.db')

with con:
    cur = con.cursor()
    cur.execute("DROP TABLE IF EXISTS Snack")
    cur.execute("CREATE TABLE Snack(Brand TEXT, Name TEXT, Price INT, url TEXT, img TEXT, Monthinfo TEXT)")
    cur.executemany("INSERT INTO Snack VALUES (?, ?, ?, ?, ?, ?)", snackList)
    
    cur.execute("DROP TABLE IF EXISTS Milkpowder")
    cur.execute("CREATE TABLE Milkpowder(Brand TEXT, Name TEXT, Price INT, url TEXT, img TEXT, Monthinfo TEXT)")
    cur.executemany("INSERT INTO Milkpowder VALUES (?, ?, ?, ?, ?, ?)", milkPowderList)
    
    cur.execute("DROP TABLE IF EXISTS Clothes")
    cur.execute("CREATE TABLE Clothes(Name TEXT, Gender TEXT, Price INT, url TEXT, img TEXT, Monthinfo TEXT)")
    cur.executemany("INSERT INTO Clothes VALUES (?, ?, ?, ?, ?, ?)", clothesList)
    
    cur.execute("DROP TABLE IF EXISTS Toy")
    cur.execute("CREATE TABLE Toy(Name TEXT, Gender TEXT, Price INT, url TEXT, img TEXT, Monthinfo TEXT)")
    cur.executemany("INSERT INTO Toy VALUES (?, ?, ?, ?, ?, ?)", toyList)
    
    cur.execute("DROP TABLE IF EXISTS Diaper")
    cur.execute("CREATE TABLE Diaper(Name TEXT, Gender TEXT, Brand TEXT, Sizeinfo TEXT, Price INT, url TEXT, img TEXT)")
    cur.executemany("INSERT INTO Diaper VALUES (?, ?, ?, ?, ?, ?, ?)", diaperList)